## PlayStore

### HMAC 키를 사용하여 Boto3로 엑세스

In [ ]:
# USE HMAC KEY TO ACCESS

import os
import boto3
from dotenv import load_dotenv

load_dotenv()

GOOGLE_ACCESS_KEY_ID = os.environ.get("GOOGLE_ACCESS_KEY_ID")
GOOGLE_ACCESS_KEY_SECRET = os.environ.get("GOOGLE_ACCESS_KEY_SECRET")

client = boto3.client(
    "s3",
    region_name="auto",
    endpoint_url="https://storage.googleapis.com",
    aws_access_key_id=GOOGLE_ACCESS_KEY_ID,
    aws_secret_access_key=GOOGLE_ACCESS_KEY_SECRET,
    config=boto3.session.Config(signature_version="s3v4"),
)

response = client.list_buckets()

### google-cloud-stroage 를 사용한 접근 방식

In [ ]:
import os
import shutil

from google.cloud import storage

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("CLIENT_SECRETS")
storage_client = storage.Client()

bucket_name = os.getenv("BUCKET_NAME")
path_download = "./data/raw/playstore"

In [ ]:
blobs = storage_client.list_blobs(bucket_name)

for blob in blobs:
    if "community" in blob.name:
        blob.download_to_filename(os.path.join(path_download, blob.name.split("/")[-1]))

In [ ]:
for csv in os.listdir(path_download):
    if "brezel" in csv:
        shutil.move(
            os.path.join(path_download, csv), os.path.join("./data/raw/brezel", csv)
        )

In [ ]:
for csv in os.listdir(os.path.join(path_download, "enta")):
    for month in ["202402", "202401"]:
        os.makedirs(f"./data/raw/enta/{month}", exist_ok=True)

        if month in csv:
            shutil.move(
                os.path.join(path_download, "enta", csv),
                os.path.join(path_download, "enta", month, csv),
            )

## AppStore

In [ ]:
import time
import os
import io
from datetime import datetime, timedelta

import jwt
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

KEY_ID = os.getenv("APPLE_KEY_ID")
ISSUER_ID = os.getenv("APPLE_ISSUER_ID")
PATH_TO_KEY = os.getenv("APPLE_PRIVATE_KEY")

IAT = int(round(time.time()))
EXP = int(round(time.time() + (10 * 60.0)))  # 10 minutes timestamp

with open(PATH_TO_KEY, "r") as f:
    PRIVATE_KEY = f.read()

headers = {"alg": "ES256", "kid": KEY_ID, "typ": "JWT"}

payload = {
    "iss": ISSUER_ID,
    "iat": IAT,
    "exp": EXP,
    "aud": "appstoreconnect-v1",
}

# Create the JWT
token = jwt.encode(payload, PRIVATE_KEY, algorithm="ES256", headers=headers)

# API Request
JWT = f"Bearer {token}"
HEAD = {"Authorization": JWT}

In [ ]:
URL = "https://api.appstoreconnect.apple.com/v1/salesReports"

list_df = []

for day in range(1, 61):
    response = requests.get(
        URL,
        params={
            "filter[frequency]": "DAILY",
            "filter[reportType]": "SALES",
            "filter[reportSubType]": "SUMMARY",
            "filter[vendorNumber]": os.getenv("APPLE_VENDOR_NUMBER"),
            "filter[reportDate]": (datetime.now() - timedelta(days=day)).strftime(
                "%Y-%m-%d"
            ),
        },
        headers=HEAD,
    )

    if response.status_code == 200:
        df = pd.read_csv(io.BytesIO(response.content), compression="gzip", sep="\t")
        list_df.append(df)
    else:
        print(response.status_code)
        print(response.json())

merge_df = pd.concat(list_df)

In [ ]:
df_enta = pd.read_csv("./data/raw/appstore/enta.csv")
df_enta

In [59]:
df_enta.loc[df_enta["Product Type Identifier"] == 3]["Units"].sum()

38

In [60]:
df_enta.loc[df_enta["Product Type Identifier"] == 1]["Units"].sum()

133

In [57]:
(
    df_enta.loc[df_enta["Product Type Identifier"] == 3]["Units"].sum()
    + df_enta.loc[df_enta["Product Type Identifier"] == 1]["Units"].sum()
)

171

In [53]:
df_enta["Product Type Identifier"].unique()

array([7, 1, 3])

### 앱스토어 리뷰 크롤링

In [ ]:
from app_store_scraper import AppStore
from pprint import pprint

minecraft = AppStore(country="kr", app_name="ENTA")
minecraft.review(how_many=20)

pprint(minecraft.reviews)
pprint(minecraft.reviews_count)